# 7 Solution Methods to Solve the Growth Model with Python

This notebook is part of a computational appendix that accompanies the paper

> MATLAB, Python, Julia: What to Choose in Economics?
> > Coleman, Lyon, Maliar, and Maliar (2017)

In order to run the codes in this notebook you will need to install and configure a few Python packages. We recommend downloading [Anaconda](https://www.continuum.io/downloads) and/or following the instructions on [quantecon.org](https://lectures.quantecon.org/py/getting_started.html). Once your Python installation is up and running, there are a few additional packages you will need in order to run the code here. To do this, you should remove the `# ` in front of the following commands and run the block below.

In [2295]:
# ! pip install git+https://github.com/EconForge/interpolation.py.git
# ! pip install quantecon
# ! pip install csaps

In [2296]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as opt
import time
import quantecon as qe
import matplotlib.pyplot as plt

from collections import namedtuple
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.misc import derivative
from interpolation.complete_poly import (CompletePolynomial,
                                         n_complete, complete_polynomial,
                                         complete_polynomial_der,
                                         _complete_poly_impl,
                                         _complete_poly_impl_vec,
                                         _complete_poly_der_impl,
                                         _complete_poly_der_impl_vec)
from numba import jit, vectorize

## Model

This section gives a short description of the commonly used stochastic Neoclassical growth model.

There is a single infinitely-lived representative agent who consumes and saves using capital. The consumer discounts the future with factor $\beta$ and derives utility from only consumption. Additionally, saved capital will depreciate at $\delta$.

The consumer has access to a Cobb-Douglas technology which uses capital saved from the previous period to produce and is subject to stochastic productivity shocks.

Productivity shocks follow an AR(1) in logs.

The agent's problem can be written recursively using the following Bellman equation


$$
\begin{align}
  V(k_t, z_t) &= \max_{k_{t+1}} u(c_t) + \beta E \left[ V(k_{t+1}, z_{t+1}) \right] \\
  &\text{subject to } \\
  c_t &= z_t f(k_t) + (1 - \delta) k_t - k_{t+1} \\
  \log z_{t+1} &= \rho \log z_t + \sigma \varepsilon
\end{align}
$$

$$
\begin{align*}
    V(X, z) &= \max_{\tilde{c},\tilde{s}} \biggl\{0,  u(c) + \beta E \bigl[V(X', z') \bigr] \biggr\}\\
    & \text{ s.t. } \tilde{c} \geq 0; \qquad \tilde{s} \geq 0 \\
    & \phantom{\text{ s.t. } }N\tilde{c} + N\tilde{s} \leq X \\
    & \phantom{\text{ s.t. } } X' = \max \biggl\{0, \bigl[X - N\tilde{c}  - N\tilde{s} - z \bigl(\frac{\tilde{c}}{\tilde{s}}\bigr)^\phi \bigr](1 + g_x) \biggr\} \\
    & \phantom{\text{ s.t. } } u(\tilde{c}) = \bar{u} + \ln(\tilde{c}) \\
    & \phantom{\text{ s.t. } } z \sim Exp( \bar{z} ) 
\end{align*} 
$$

## Python Code

We begin by defining a `namedtuple` that contains the parameters of our model. This is useful to pass the parameters around to functions that are just-in-time (JIT) compiled by `numba`.

In [2297]:
#
# Create a named tuple type that we can pass into the jitted functions
# so that we don't have to pass parameters one by one
#
Params = namedtuple("Params", ["N", "beta", "g", "a_bar", "eta", "u_bar"])

@jit(nopython=True)
def param_unpack(params):
    "Unpack parameters from the Params type"
    out = (params.N, params.beta, params.g,
           params.a_bar, params.eta, params.u_bar)

    return out

We will then define various helper functions to ensure that we [don't repeat ourselves](https://lectures.quantecon.org/py/writing_good_code.html#don-t-repeat-yourself) and that the inner functions can be JIT compiled.

In [2298]:
#
# Helper functions to make sure things are jitted
#
@vectorize(nopython=True)
def u(c, eta, u_bar):
    "log utility function"
    return -1e10 if c < 1e-10 else (u_bar + np.log(c)) if eta==1 else (u_bar + (c**(1-eta)/(1-eta)))

@vectorize(nopython=True)
def du(c,eta):
    "Derivative of log utility function"
    return 1e10 if c < 1e-10 else c**(-eta)

@vectorize(nopython=True)
def duinv(u,eta):
    "Inverse of the derivative of the CRRA utility function"
    return u**(-1.0/eta )

@vectorize(nopython=True)
def f(X):
    "production function"
    return X

@vectorize(nopython=True)
def df(X):
    "Derivative of production function"
    return 1

@vectorize(nopython=True)
def discount_factor(beta, a_bar, c, w):
    "risk adjusted discount factor"
    return beta*a_bar*w/(c + a_bar*w)

@vectorize(nopython=True)
def next_x(X, N, g, c, w):
    # next period's CPR given c and w
    return (X - N*(c + w))*(1+g)

@vectorize(nopython=True)
def vector_dampen(vec1, vec2, dampen=1):
    return dampen*vec1 + (1-dampen)*vec2

@vectorize(nopython=True)
def expendables(X, Xp, N, g):
    # get expendables given X and X prime
    return (1/N)*(X - Xp/(1 + g))

@vectorize(nopython=True)
def c_from_X_Xp(X, Xp, Vp, beta, a):
    # get expendables given X and X prime
    return -(2*E*a - 2*E*a**2 + E*a*(Vp*beta*(4*a + Vp*beta - 4))**0.5 - E*Vp*a*beta)/(2*(a**2 - 2*a + 1))

@vectorize(nopython=True)
def c_from_exp(Vp, E, beta, a_bar):
    if Vp <= 0: 
        return E
    return -(2*E*a_bar - 2*E*a_bar**2 + E*a_bar*(Vp*beta*(4*a_bar + Vp*beta - 4))**0.5 - E*Vp*a_bar*beta)/(2*(a_bar**2 - 2*a_bar + 1))



def x_curve(x, a, b):
    return a + b*x


def v_curve(x, a, b, c, d):
    return a*np.log(x)/(c - (x**b)) + d


@vectorize(nopython=True)
def trunc_to_one(X):
    return X if X >= 1 else 1

@vectorize(nopython=True)
def psurvival(a_bar, c, w):
    return a_bar*w/(c + a_bar*w)


We also now define a class that contains

1. Parameters of the growth model
2. Grids used for approximating the solution
3. Nodes and weights used to approximate integration

This again helps us maintain all of the relevant information in one place and simplifies passing it to other functions.

In [2299]:
class Model(object):
    """
    The stochastic Neoclassical Growth model contains
    parameters which include

    * alpha: Capital share in output
    * beta: discount factor
    * delta: depreciation rate
    * gamma: risk aversion
    * rho: persistence of the log productivity level
    * sigma: standard deviation of shocks to log productivity
    """
    def __init__(self, N=1, beta=0.95, g_x=0.05, g_a=0.015, a_bar=1e5, 
                 eta=1, u_bar=0, 
                 xmin_l=2, xmax_l=200, nx_l=25, linspace_l=True,
                 xmin_h=250, xmax_h=2500, nx_h=100, linspace_h=False):
        # calculate g from g_a and g_x 
        g = g_a + g_x + g_a*g_x

        # Household parameters
        self.beta, self.eta, self.u_bar, = beta, eta, u_bar

        # Firm/technology parameters
        self.N, self.g, self.a_bar = N, g, a_bar

        # Create t grids
        if linspace_l:
            self.xgrid_l = N*np.linspace(xmin_l, xmax_l, nx_l)
        else:
            self.xgrid_l = N*np.geomspace(xmin_l, xmax_l, nx_l)
            
        if linspace_h:
            self.xgrid_h = N*np.linspace(xmin_h, xmax_h, nx_h)
        else:
            self.xgrid_h = N*np.geomspace(xmin_h, xmax_h, nx_h)
            
        self.xgrid = np.concatenate([self.xgrid_l, self.xgrid_h])
        # for splitting spline versus nonlin policy functions
        self.xsplit = self.xgrid_l[-1]
        self.xsplit_idx = nx_l
        
        self.ns = nx_l + nx_h
        
    def _unpack_params(self):
        out = (self.N, self.beta, self.g, 
               self.a_bar, self.eta, self.u_bar)
        return out

    def _unpack_grids(self):
        out = (self.xgrid)
        return out


## Solution Methods

In this notebook, we describe the following solution methods:

* Conventional Value Function Iteration
* Envelope Condition Value Function Iteration
* Envelope Condition Derivative Value Function Iteration
* Endogenous Grid Value Function Iteration
* Conventional Policy Function Iteration
* Envelope Condition Policy Function Iteration
* Euler Equation Method

Each of these solution methods will have a very similar structure that follows a few basic steps:

1. Guess a function (either value function or policy function).
2. Using this function, update our guess of both the value and policy functions.
3. Check whether the function we guessed and what it was updated to are similar enough. If so, proceed. If not, return to step 2 using the updated functions.
4. Output the policy and value functions.

In order to reduce the amount of repeated code and keep the exposition as clean as possible (the notebook is plenty long as is...), we will define a class for each solution method that inherit various properties from a general solution parent class. The parent class will contain methods which apply to all of the other solution methods such as a general solve method, computing expectations, simulating, etc... This implementation may feel strange at first if one isn't familiar with object oriented programming, but these concepts can be powerful when properly used.

In [2300]:
class GeneralSolution:
    """
    This is a general solution method. We define this, so that we can
    sub-class it and define specific update methods for each particular
    solution method
    """
    def __init__(self, ncgm, prev_sol=None, maxfev=10000, interpolator='linear'):
        # Save model and approximation degree
        self.ncgm,  self.maxfev, self.interpolator = ncgm, maxfev, interpolator

        # Unpack some info from ncgm
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        X = self.ncgm.xgrid
        self.xsplit = self.ncgm.xsplit
        self.xsplit_idx = self.ncgm.xsplit_idx
        

        # Use parameter values from model to create a namedtuple with
        # parameters saved inside
        self.params = Params(N, beta, g, a_bar, eta, u_bar)
        
        # initialize CP and WP as None:
        self.CP = None
        self.WP = None

        # Update to fill initial value and policy matrices
        # If we give it another solution type then use it to
        # generate values and policies
        if issubclass(type(prev_sol), GeneralSolution):
            self.XP = prev_sol.XP
            self.VF = prev_sol.VF
        # If we give it a tuple then assume it is (policy, value) pair
        elif type(prev_sol) is tuple:
            self.XP = prev_sol[0]
            self.VF = prev_sol[1]
        # Otherwise guess a constant value function and a policy
        # of roughly steady state
        else:
            # guess VF and XP
            self.VF = np.log(X/N) 
            self.XP = X

        # Fit spline based on guesses
        self.x_spline = interp1d(X[:self.xsplit_idx], self.XP[:self.xsplit_idx], kind=self.interpolator, fill_value='extrapolate')
        self.v_spline = interp1d(X[:self.xsplit_idx], self.VF[:self.xsplit_idx], kind=self.interpolator, fill_value='extrapolate')
        
        # Params based on guesses
        self.x_coeffs, _ = opt.curve_fit(x_curve
                                         , X[self.xsplit_idx:]
                                         , self.XP[self.xsplit_idx:]
                                         , maxfev=self.maxfev)
        self.v_coeffs, _ = opt.curve_fit(v_curve
                                         , X[self.xsplit_idx:]
                                         , self.VF[self.xsplit_idx:]
                                         , maxfev=self.maxfev)
  
    def build_VF(self, X=None):
        """
        Using the current coefficients, this builds the value function
        for all states
        """
        if X is None:
            X = self.ncgm.xgrid
        
        VF = np.where(X > self.xsplit
                        , v_curve(X, *self.v_coeffs)
                        , self.v_spline(X))

        return VF
    
    def build_XP(self, X=None):
        if X is None:
            X = self.ncgm.xgrid
        
        XP = np.where(X > self.xsplit
                        , np.clip(x_curve(X, *self.x_coeffs)
                                  , 0, X*(1 + self.ncgm.g))
                        , self.x_spline(X))
        
        return XP
    
    def compute_splines(self, XP, VF):
        if isinstance(self, IterateOnPolicy):
            new_x_spline = self.x_spline
        else:
            new_x_spline = interp1d(self.ncgm.xgrid[:self.xsplit_idx]
                                    , XP[:self.xsplit_idx]
                                    , kind=self.interpolator
                                    , fill_value='extrapolate')
        
        new_v_spline = interp1d(self.ncgm.xgrid[:self.xsplit_idx]
                                , VF[:self.xsplit_idx]
                                , kind=self.interpolator
                                , fill_value='extrapolate')
        
        return new_x_spline, new_v_spline
    

#         # Params based on guesses
#         self.x_coeffs, _ = opt.curve_fit(x_curve, X, self.XP, maxfev=self.maxfev)
#         self.v_coeffs, _ = opt.curve_fit(v_curve, X, self.VF, maxfev=self.maxfev)
        
#     def build_VF(self, X=None):
#         """
#         Using the current coefficients, this builds the value function
#         for all states
#         """
#         if X is None:
#             X = self.ncgm.xgrid
            
#         VF = v_curve(X, *self.v_coeffs)

#         return VF
    
#     def build_XP(self, X=None):
#         if X is None:
#             X = self.ncgm.xgrid
        
#         XP = np.clip(x_curve(X, *self.x_coeffs), 0, X*(1 + self.ncgm.g))
        
#         return XP
    
    def compute_coeffs(self, XP, VF):
        if isinstance(self, IterateOnPolicy):
            new_x_coeffs = self.x_coeffs
        else:
            new_x_coeffs, _ = opt.curve_fit(x_curve
                                            , self.ncgm.xgrid[self.xsplit_idx:]
                                            , XP[self.xsplit_idx:]
                                            , maxfev=self.maxfev)
        
        new_v_coeffs, _ = opt.curve_fit(v_curve
                                        , self.ncgm.xgrid[self.xsplit_idx:]
                                        , VF[self.xsplit_idx:]
                                        , maxfev=self.maxfev) 
        
        return new_x_coeffs, new_v_coeffs

    def _unpack_params(self):
        return self.ncgm._unpack_params()

    def build_CP(self, E=None, VP=None, X=None, XP=None):
        """
        Using the current coefficients, this builds the policy function
        for all states
        """
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        
        # Check if expected value next period (EVP) and expendables (E) are given
        if VP is None or E is None:
            if X is None:
                X = self.ncgm.xgrid
                
            if XP is None:
                XP = self.build_XP(X)
                
            VP = self.compute_EV(XP)
            E = expendables(X, XP, N, g)
            
        CP = np.clip(c_from_exp(VP, E, beta, a_bar), 0, np.maximum(E, 0))

        return CP
    
    def build_WP(self, CP=None, E=None, VP=None, X=None, XP=None):
        """
        Using the current coefficients, this builds the policy function
        for all states
        """
        if CP is None or E is None:
            N, beta, g, a_bar, eta, u_bar = self._unpack_params()
            if VP is None or E is None:
                if X is None:
                    X = self.ncgm.xgrid

                if XP is None:
                    XP = self.build_XP(X)

                VP = self.compute_EV(XP)
                E = expendables(X, XP, N, g)

                CP = np.clip(c_from_exp(VP, E, beta, a_bar), 0, X/N)
            else:
                CP = self.build_CP(E=E, VP=VP)
            
        return E - CP

    def update_v(self, new_v_coeffs, new_v_spline):
        """
        Updates the coefficients for the value function
        """
        self.v_coeffs = new_v_coeffs
        self.v_spline = new_v_spline

        return None

    def update_x(self, new_x_coeffs, new_x_spline):
        """
        Updates the coefficients for the policy function
        """
        self.x_coeffs = new_x_coeffs
        self.x_spline = new_x_spline
        
        return None

    def update(self):
        """
        Given the current state of everything in solution, update the
        value and policy coefficients
        """
        emsg = "The update method is implemented in solution specific classes"
        emsg += "\nand cannot be called from `GeneralSolution`"
        raise ValueError(emsg)


    def compute_EV(self, XP=None):
        """
        Compute the expected value
        """
        if XP is None:
            # Use policy to compute XP
            XP = self.build_XP()
        
        EV = self.build_VF(XP) 
        
        return EV


    def compute_distance(self, XP, VF):
        """
        Computes average distance between policy functions
        """
        return np.max(np.abs(1.0 - (XP+1e-12)/(self.XP+1e-12)))
    

    def solve(self, tol=1e-6, maxiter=2500, verbose=False, nskipprint=25):
        # Iterate until convergence
        dist, it = 10.0, 0
        while (dist>tol) and (it<maxiter):
            # Run solution specific update code
            XP, VF = self.update()
            

            # Compute new policy and value coeffs
            new_x_coeffs,  new_v_coeffs = self.compute_coeffs(XP, VF)
            new_x_spline, new_v_spline = self.compute_splines(XP, VF)

            # Update distance and iterations
            dist = self.compute_distance(XP, VF)
            self.XP, self.VF = XP, VF
            it += 1
            if verbose and it%nskipprint == 0:
                print(it, dist)

            # Update all coefficients
            self.update_x(new_x_coeffs, new_x_spline)
            self.update_v(new_v_coeffs, new_v_spline)

        # After finishing iteration, iterate to convergence using policy
        if not isinstance(self, IterateOnPolicy):
            sol_iop = IterateOnPolicy(self.ncgm, prev_sol=self, maxfev=self.maxfev)
            XP, VF = sol_iop.solve(tol=1e-6)

            # Save final versions of everything
            self.XP, self.VF = XP, VF
            
            new_x_coeffs, new_v_coeffs = sol_iop.compute_coeffs(XP, VF)
            new_x_spline, new_v_spline = sol_iop.compute_splines(XP, VF)
            self.update_x(new_x_coeffs, new_x_spline)
            self.update_v(new_v_coeffs, new_v_spline)

        return self.XP, self.VF
        

    def simulate(self, X=None, T=1000, s=1):
        """
        Simulates the neoclassical growth model with policy function
        given by self.KP. Simulates for `T` periods and discarsd first
        nburn `observations`
        """
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        # X is starting point, if None, set to N*274
        if X is None:
            X = N*274
        
        def recurse_bellman(x, t):
            if x <= 0:
                return 0, 0, x, t
            if t >= T:
                return u(x/N, 1, u_bar), 1, x, T
            c_corner = x/N 
            u_corner = u(c_corner, 1, u_bar)
            if u_corner <= 0:
                return 0, 0, x, t
            
            xp = self.build_XP(x)
            if xp < 0:
                return u_corner, 0, 0, t+1
            
            if s != 1:
                xp = s*xp
            
            e = expendables(x, xp, N, g)
            if e <= 0:
                return u_corner, 0, 0, t+1
            evp = self.compute_EV(xp)
            
            c = self.build_CP(VP=evp, E=e)
            w = e - c
            
            m_t = psurvival(a_bar, c, w)
            u_t = u(c, 1, u_bar)
            
            vp, mp, final_state, nperiods = recurse_bellman(xp, t+1)            
            
            if u_t + beta*m_t*vp <= u_corner:
                return u_corner, 0, 0, t+1
            
            return u_t + beta*m_t*vp, m_t*mp, final_state, nperiods
        
        return recurse_bellman(X, 0)


### Iterating to Convergence (given policy)

This isn't one of the methods described above, but it is used as an element of a few of our methods (and also as a way to get a first guess at the value function). This method takes an initial policy function, $\bar{k}(k_t, z_t)$, as given, and then, without changing the policy, iterates until the value function has converged.

Thus the "update section" of the algorithm in this instance would be:

* Leave policy function unchanged
* At each point of grid, $(k_t, z_t)$, compute $\hat{V}(k_t, z_t) = u(c(\bar{k}(k_t, z_t))) + \beta E \left[ V(\bar{k}(k_t, z_t), z_{t+1}) \right]$

We override two of the methods from `GeneralSolution`

* `compute_distance` because when we are iterating to convergence on the value function we want to check distnace using value function rather than policy function
* `compute_coefficients` because we don't need to update the policy functions coefficients.

The `update` method just repeatedly applies a particular policy function to update the value function.

In [2301]:
class IterateOnPolicy(GeneralSolution):
    """
    Subclass of the general solution method. The update method for this
    class simply computes the fixed point of the value function given
    a specific policy
    """
    def compute_distance(self, XP, VF):
        """
        Computes distance between policy functions. When we are
        iterating on a specific policy, we would like to compute
        distances by the difference between VFs
        """
        return np.max(np.abs(1.0 - (VF )/(self.VF )))
    
    def update(self):
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        X = self.ncgm.xgrid
        
        VP = self.compute_EV(self.XP)
        E = expendables(X, self.XP, N, g)
        CP = self.build_CP(VP=VP, E=E)
        WP = E - CP
        
        # Update the value function
        VF = u(CP, eta, u_bar) + beta*psurvival(a_bar, CP, WP)*np.maximum(VP, np.zeros(self.ncgm.ns))
        VF = np.where(VF < 0, 0, VF)
        
        return self.XP, VF

In [2302]:
# class Nonlinear(GeneralSolution):
#     """
#     Subclass of the general solution method. The update method for this
#     class simply computes the fixed point of the value function given
#     a specific policy
#     """
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
        
#         # Params based on guesses
#         self.x_coeffs, _ = opt.curve_fit(x_curve, self.ncgm.xgrid, self.XP, maxfev=self.maxfev)
#         self.v_coeffs, _ = opt.curve_fit(v_curve, self.ncgm.xgrid, self.VF, maxfev=self.maxfev)
    
#     def compute_coeffs(self, XP, VF):
#         if isinstance(self, IterateOnPolicy):
#             new_x_coeffs = self.x_coeffs
#         else:
#             new_x_coeffs, _ = opt.curve_fit(x_curve, self.ncgm.xgrid, XP, maxfev=self.maxfev)
        
#         new_v_coeffs, _ = opt.curve_fit(v_curve, self.ncgm.xgrid, VF, maxfev=self.maxfev) 
        
#         return new_x_coeffs, new_v_coeffs
    
#     def build_VF(self, X=None):
#         """
#         Using the current coefficients, this builds the value function
#         for all states
#         """
#         if X is None:
#             X = self.ncgm.xgrid
            
#         VF = v_curve(X, *self.v_coeffs)

#         return VF
    
#     def build_XP(self, X=None):
#         if X is None:
#             X = self.ncgm.xgrid
        
#         XP = np.clip(x_curve(X, *self.x_coeffs), 0, X*(1 + self.ncgm.g))
        
#         return XP
    
    
# class IterateOnPolicy(Nonlinear):
#     """
#     Subclass of the general solution method. The update method for this
#     class simply computes the fixed point of the value function given
#     a specific policy
#     """
#     def compute_distance(self, XP, VF):
#         """
#         Computes distance between policy functions. When we are
#         iterating on a specific policy, we would like to compute
#         distances by the difference between VFs
#         """
#         return np.max(np.abs(1.0 - (VF )/(self.VF )))
    
#     def update(self):
#         # Unpack parameters
#         N, beta, g, a_bar, eta, u_bar = self._unpack_params()
#         X = self.ncgm.xgrid
        
#         if self.CP is None or self.WP is None:
#             VP = self.compute_EV(self.XP)
#             E = expendables(X, self.XP, N, g)
#             self.CP = self.build_CP(VP=VP, E=E)
#             self.WP = E - self.CP
        
#         # Update the value function
#         VF = u(self.CP, eta, u_bar) + discount_factor(beta, a_bar, self.CP, self.WP)*np.maximum(self.compute_EV(self.XP), np.zeros(self.ncgm.ns))

#         return self.XP, VF

In [2303]:
# class Spline(GeneralSolution):
#     """
#     Subclass of the general solution method. The update method for this
#     class simply computes the fixed point of the value function given
#     a specific policy
#     """
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
        
#         # Fit spline based on guesses
#         self.x_coeffs = interp1d(X, self.XP, kind=self.interpolator)
#         self.v_coeffs = interp1d(X, self.VF, kind=self.interpolator)
        
        
#     def compute_coeffs(self, XP, VF):
#         if isinstance(self, IterateOnPolicy):
#             new_x_coeffs = self.x_coeffs
#         else:
#             new_x_coeffs = interp1d(X, self.XP, kind=self.interpolator)
        
#         new_v_coeffs = interp1d(X, self.VF, kind=self.interpolator)
        
#         return new_x_coeffs, new_v_coeffs
    
    
#     def build_VF(self, X=None):
#         """
#         Using the current coefficients, this builds the value function
#         for all states
#         """
#         if X is None:
#             X = self.ncgm.xgrid
            
#         VF = self.v_coeffs(X) 

#         return VF
    
#     def build_XP(self, X=None):
#         if X is None:
#             X = self.ncgm.xgrid
        
#         XP = self.x_coeffs(X) 
        
#         return XP
    
    
# class IterateOnPolicy(Spline):
#     """
#     Subclass of the general solution method. The update method for this
#     class simply computes the fixed point of the value function given
#     a specific policy
#     """
#     def compute_distance(self, XP, VF):
#         """
#         Computes distance between policy functions. When we are
#         iterating on a specific policy, we would like to compute
#         distances by the difference between VFs
#         """
#         return np.max(np.abs(1.0 - (VF )/(self.VF )))
    
#     def update(self):
#         # Unpack parameters
#         N, beta, g, a_bar, eta, u_bar = self._unpack_params()
#         X = self.ncgm.xgrid
        
#         if self.CP is None or self.WP is None:
#             VP = self.compute_EV(self.XP)
#             E = expendables(X, self.XP, N, g)
#             self.CP = self.build_CP(VP=VP, E=E)
#             self.WP = E - self.CP
        
#         # Update the value function
#         VF = u(self.CP, eta, u_bar) + discount_factor(beta, a_bar, self.CP, self.WP)*np.maximum(self.compute_EV(self.XP), np.zeros(self.ncgm.ns))

#         return self.XP, VF

### Conventional Value Function Iteration

This is one of the first solution methods for macroeconomics a graduate student in economics typically learns.

In this solution method, one takes as given a value function, $V(k_t, z_t)$, and then solves for the optimal policy given the value function.

The update section takes the form:

* For each point, $(k_t, z_t)$, numerically solve for $c^*(k_t, z_t)$ to satisfy the first order condition $u'(c^*) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$


* For each point, $(X_t, z_t)$, numerically solve for $c^*(X_t, z_t)$ to satisfy the first order condition $u'(\tilde{c}^\ast) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$




In [2304]:
class VFI(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def update(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        xgrid = self.ncgm.xgrid
        n_state = xgrid.shape[0]

        # Get the policy and update it
        XP = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current vals
            X = xgrid[i_s]
            def _f(_xp):
                EV = self.compute_EV(_xp)
                E = expendables(X, _xp, N, g)
                _vf_corner = max(u(X/N, 1, u_bar),0)
                
                if E <= 0:
                    return -_vf_corner**2
                
                _cp = self.build_CP(VP=EV, E=E)
#                 _cp = opt.fminbound(lambda t: -(u(t, 1, u_bar) + discount_factor(beta, a_bar, t, E-t)*EV)**2,
#                                         0, E, xtol=1e-4)
                _wp = E - _cp
                _vf = u(_cp, 1, u_bar) + beta*psurvival(a_bar, _cp, _wp)*EV
                
                if _vf < _vf_corner:
                    return -_vf_corner**2
                
                return -_vf**2
                
            
            _xp = opt.fminbound(_f, 0, X*(1+g), xtol=1e-12)
            EV = self.compute_EV(_xp)
            E = expendables(X, _xp, N, g)
            _vf_corner = max(u(X/N, 1, u_bar),0)
            if E <= 0:
                _xp = 0
                _vf = _vf_corner
            else:
                cp = self.build_CP(VP=EV, E=E)
                wp = E - cp
                
                _vf = u(cp, 1, u_bar) + beta*psurvival(a_bar, cp, wp)*EV
             
            if _vf < _vf_corner:
                XP[i_s] = 0
                VF[i_s] = _vf_corner
            else:                
                XP[i_s] = _xp
                VF[i_s] = _vf

        return XP, VF


## A Horse Race

We can now run a horse race to compare the methods in terms of both accuracy and speed.

In [2305]:
ncgm = Model(N=1, a_bar=1e5, beta=0.95, g_x=0.05, 
             xmin_l=2, xmax_l=200, nx_l=25, linspace_l=True,
             xmin_h=250, xmax_h=2500, nx_h=100, linspace_h=False
            )

# First guess
vp = IterateOnPolicy(ncgm, maxfev=50000, interpolator='linear')
vp.solve(tol=1e-9)

np.random.seed(61089)



print('vp done')
new_sol = vp
new_sol = VFI(ncgm, maxfev=100000, prev_sol=vp, interpolator='linear')
# new_sol.XP = 1.5*new_sol.XP
# new_sol = VFI(ncgm, maxfev=100000, prev_sol=new_sol, interpolator='linear')
ts = time.time()
new_sol.solve(tol=1e-6, verbose=True, nskipprint=25)
time_took = time.time() - ts

/Users/mikemahon/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
<ipython-input-2301-a084eb813900>:13: RuntimeWarning: invalid value encountered in true_divide
  return np.max(np.abs(1.0 - (VF )/(self.VF )))


vp done
25 0.0013743583009531157
50 0.0003363687528816772
75 8.920300225967459e-05
100 2.4607960511735527e-05
125 6.898878725403179e-06
150 1.9584878727663124e-06


<ipython-input-2298-8e8cfbd80eb2>:66: RuntimeWarning: divide by zero encountered in log
  return a*np.log(x)/(c - (x**b)) + d
<ipython-input-2298-8e8cfbd80eb2>:66: RuntimeWarning: divide by zero encountered in power
  return a*np.log(x)/(c - (x**b)) + d
<ipython-input-2298-8e8cfbd80eb2>:66: RuntimeWarning: invalid value encountered in true_divide
  return a*np.log(x)/(c - (x**b)) + d


In [2306]:
new_sol.VF

array([  0.69314718,   3.81581542,   7.09076876,  10.62591418,
        15.04738869,  19.26925519,  22.75430241,  25.72066576,
        28.30242247,  30.58774361,  32.63766699,  34.49618245,
        36.19603666,  37.76226421,  39.21443496,  40.56813923,
        41.83600104,  43.0283886 ,  44.1539242 ,  45.21985794,
        46.23234663,  47.1966652 ,  48.1173694 ,  48.99842258,
        49.84329557,  54.27494961,  54.73686904,  55.19878927,
        55.66071031,  56.12263212,  56.58455472,  57.04647808,
        57.5084022 ,  57.97032707,  58.43225269,  58.89417903,
        59.3561061 ,  59.81803388,  60.27996237,  60.74189155,
        61.20382143,  61.66575199,  62.12768322,  62.58961513,
        63.05154769,  63.51348091,  63.97541477,  64.43734927,
        64.8992844 ,  65.36122016,  65.82315654,  66.28509353,
        66.74703113,  67.20896932,  67.67090811,  68.13284749,
        68.59478745,  69.05672798,  69.51866909,  69.98061076,
        70.44255299,  70.90449577,  71.3664391 ,  71.82

In [2307]:
new_sol.v_coeffs

array([-3.13568236e+05, -2.25732749e+02, -1.57876915e+04, -5.53903368e+01])

In [2308]:
new_sol.build_VF()

array([  0.69314718,   3.81581542,   7.09076876,  10.62591418,
        15.04738869,  19.26925519,  22.75430241,  25.72066576,
        28.30242247,  30.58774361,  32.63766699,  34.49618245,
        36.19603666,  37.76226421,  39.21443496,  40.56813923,
        41.83600104,  43.0283886 ,  44.1539242 ,  45.21985794,
        46.23234663,  47.1966652 ,  48.1173694 ,  48.99842258,
        49.84329557,  54.27450966,  54.73645855,  55.19840744,
        55.66035633,  56.12230522,  56.58425411,  57.04620299,
        57.50815188,  57.97010077,  58.43204966,  58.89399855,
        59.35594744,  59.81789633,  60.27984521,  60.7417941 ,
        61.20374299,  61.66569188,  62.12764077,  62.58958966,
        63.05153855,  63.51348744,  63.97543632,  64.43738521,
        64.8993341 ,  65.36128299,  65.82323188,  66.28518077,
        66.74712966,  67.20907854,  67.67102743,  68.13297632,
        68.59492521,  69.0568741 ,  69.51882299,  69.98077188,
        70.44272077,  70.90466965,  71.36661854,  71.82

In [2309]:
N, beta, g, a_bar, eta, u_bar = new_sol._unpack_params()
X = new_sol.ncgm.xgrid
VF = new_sol.VF
XP = new_sol.XP
CP = new_sol.build_CP()
WP = new_sol.build_WP()

<ipython-input-2298-8e8cfbd80eb2>:66: RuntimeWarning: divide by zero encountered in log
  return a*np.log(x)/(c - (x**b)) + d
<ipython-input-2298-8e8cfbd80eb2>:66: RuntimeWarning: divide by zero encountered in power
  return a*np.log(x)/(c - (x**b)) + d
<ipython-input-2298-8e8cfbd80eb2>:66: RuntimeWarning: invalid value encountered in true_divide
  return a*np.log(x)/(c - (x**b)) + d


In [2310]:
X

array([2.00000000e+00, 1.02500000e+01, 1.85000000e+01, 2.67500000e+01,
       3.50000000e+01, 4.32500000e+01, 5.15000000e+01, 5.97500000e+01,
       6.80000000e+01, 7.62500000e+01, 8.45000000e+01, 9.27500000e+01,
       1.01000000e+02, 1.09250000e+02, 1.17500000e+02, 1.25750000e+02,
       1.34000000e+02, 1.42250000e+02, 1.50500000e+02, 1.58750000e+02,
       1.67000000e+02, 1.75250000e+02, 1.83500000e+02, 1.91750000e+02,
       2.00000000e+02, 2.50000000e+02, 2.55882755e+02, 2.61903938e+02,
       2.68066806e+02, 2.74374691e+02, 2.80831008e+02, 2.87439249e+02,
       2.94202988e+02, 3.01125885e+02, 3.08211685e+02, 3.15464221e+02,
       3.22887416e+02, 3.30485287e+02, 3.38261944e+02, 3.46221593e+02,
       3.54368541e+02, 3.62707195e+02, 3.71242066e+02, 3.79977771e+02,
       3.88919036e+02, 3.98070698e+02, 4.07437709e+02, 4.17025134e+02,
       4.26838162e+02, 4.36882100e+02, 4.47162382e+02, 4.57684570e+02,
       4.68454356e+02, 4.79477565e+02, 4.90760163e+02, 5.02308251e+02,
      

In [2311]:
VF

array([  0.69314718,   3.81581542,   7.09076876,  10.62591418,
        15.04738869,  19.26925519,  22.75430241,  25.72066576,
        28.30242247,  30.58774361,  32.63766699,  34.49618245,
        36.19603666,  37.76226421,  39.21443496,  40.56813923,
        41.83600104,  43.0283886 ,  44.1539242 ,  45.21985794,
        46.23234663,  47.1966652 ,  48.1173694 ,  48.99842258,
        49.84329557,  54.27494961,  54.73686904,  55.19878927,
        55.66071031,  56.12263212,  56.58455472,  57.04647808,
        57.5084022 ,  57.97032707,  58.43225269,  58.89417903,
        59.3561061 ,  59.81803388,  60.27996237,  60.74189155,
        61.20382143,  61.66575199,  62.12768322,  62.58961513,
        63.05154769,  63.51348091,  63.97541477,  64.43734927,
        64.8992844 ,  65.36122016,  65.82315654,  66.28509353,
        66.74703113,  67.20896932,  67.67090811,  68.13284749,
        68.59478745,  69.05672798,  69.51866909,  69.98061076,
        70.44255299,  70.90449577,  71.3664391 ,  71.82

In [2312]:
XP

array([   0.        ,    8.13772388,   17.06130953,   26.04980644,
         35.24194501,   43.59932059,   51.95582463,   60.3119085 ,
         68.66779887,   77.02364034,   85.37954272,   93.735609  ,
        102.09194119,  110.44865247,  118.8058671 ,  127.1637241 ,
        135.52237898,  143.88200506,  152.24279474,  160.604959  ,
        168.9687277 ,  177.33434876,  185.7020869 ,  194.07222193,
        202.41543495,  253.01953644,  258.97336884,  265.06730125,
        271.30463039,  277.68873052,  284.22305534,  290.91113977,
        297.75660194,  304.76314513,  311.93455974,  319.27472536,
        326.78761289,  334.47728667,  342.34790664,  350.40373067,
        358.64911681,  367.08852564,  375.72652275,  384.5677811 ,
        393.61708365,  402.87932592,  412.35951859,  422.06279025,
        431.99439021,  442.15969126,  452.56419262,  463.21352295,
        474.11344332,  485.2698504 ,  496.68877954,  508.37640822,
        520.3390669 ,  532.58320402,  545.11546655,  557.94262

In [2313]:
CP

array([  2.        ,   2.60039075,   2.4717602 ,   2.28458436,
         1.90921949,   2.30893093,   2.70927421,   3.10982283,
         3.51037754,   3.91081887,   4.31105924,   4.71101618,
         5.11060669,   5.50973622,   5.90829889,   6.3061743 ,
         6.70322609,   7.09930089,   7.49422721,   7.88781608,
         8.27986078,   8.6701377 ,   9.05840757,   9.44441703,
         9.8550925 ,  12.30739958,  12.59574934,  12.89086108,
        13.19289342,  13.50200871,  13.81837311,  14.14215668,
        14.47353348,  14.81268163,  15.15978345,  15.51502552,
        15.87859884,  16.25069883,  16.63152555,  17.02128372,
        17.42018289,  17.82843751,  18.24626706,  18.6738962 ,
        19.11155481,  19.55947822,  20.01790725,  20.48708838,
        20.96727387,  21.45872191,  21.96169674,  22.4764688 ,
        23.00331488,  23.54251828,  24.09436893,  24.65916356,
        25.23720588,  25.82880672,  26.43428421,  27.05396394,
        27.68817914,  28.33727086,  29.00158817,  29.68

In [2314]:
WP

array([0.        , 0.0139312 , 0.01950462, 0.02271666, 0.02303948,
       0.0315494 , 0.04024525, 0.04913003, 0.05819025, 0.06740976,
       0.07677306, 0.08626602, 0.09587589, 0.10559104, 0.11540076,
       0.125295  , 0.13526419, 0.1452991 , 0.15539067, 0.16552992,
       0.17570789, 0.18591554, 0.19614378, 0.20638341, 0.21721812,
       0.28315776, 0.29103168, 0.29911407, 0.30741033, 0.31592598,
       0.3246667 , 0.3336383 , 0.34284675, 0.35229817, 0.36199882,
       0.37195514, 0.38217372, 0.39266132, 0.40342487, 0.41447148,
       0.42580845, 0.43744324, 0.44938353, 0.46163716, 0.47421219,
       0.48711689, 0.50035973, 0.51394939, 0.52789478, 0.54220502,
       0.55688949, 0.57195777, 0.58741971, 0.60328541, 0.6195652 ,
       0.63626969, 0.65340976, 0.67099656, 0.68904152, 0.70755636,
       0.7265531 , 0.74604406, 0.76604187, 0.78655948, 0.80761016,
       0.82920753, 0.85136555, 0.87409851, 0.89742109, 0.92134833,
       0.94589564, 0.97107882, 0.99691408, 1.02341802, 1.05060

In [2315]:
(XP - X)/X

array([-1.        , -0.20607572, -0.07776705, -0.02617546,  0.00691271,
        0.00807678,  0.00885096,  0.00940433,  0.00982057,  0.0101461 ,
        0.01040879,  0.01062651,  0.0108113 ,  0.01097165,  0.01111376,
        0.01124234,  0.01136104,  0.01147279,  0.01158003,  0.01168478,
        0.01178879,  0.01189357,  0.01200047,  0.01211068,  0.01207717,
        0.01207815,  0.01207824,  0.01207833,  0.01207843,  0.01207852,
        0.01207861,  0.01207869,  0.01207878,  0.01207887,  0.01207895,
        0.01207904,  0.01207912,  0.01207921,  0.01207929,  0.01207937,
        0.01207945,  0.01207953,  0.0120796 ,  0.01207968,  0.01207976,
        0.01207983,  0.01207991,  0.01207998,  0.01208005,  0.01208013,
        0.0120802 ,  0.01208027,  0.01208034,  0.01208041,  0.01208048,
        0.01208055,  0.01208063,  0.01208068,  0.01208075,  0.01208081,
        0.01208087,  0.01208094,  0.012081  ,  0.01208107,  0.01208113,
        0.01208119,  0.01208125,  0.01208131,  0.01208137,  0.01

In [ ]:
for x in range(10,100, 10):
    print('x/N: ', x/N)
    print('VF estimate: ', new_sol.build_VF(x))
    sim_res1 = new_sol.simulate(x,2000,s=1)
    print('new_sol.simulate(x,2000,s=1): ', sim_res1)
    sim_res1 = sim_res1[0]
    print('sim_res1 (s=1): ', sim_res1)
    sim_res2 = new_sol.simulate(x,2000,s=0.9)[0]
    print('sim_res2 (s=0.9): ', sim_res2)
    sim_res3 = new_sol.simulate(x,2000,s=1.1)[0]
    print('sim_res3 (s=1.1): ', sim_res3)
    print('sim_res2 < sim_res1 > sim_res3???: ', sim_res1 >= sim_res2 and sim_res1 >= sim_res3)
    print('')

x/N:  10.0
VF estimate:  3.721189110861415
new_sol.simulate(x,2000,s=1):  (3.6561642962057457, 0.0, 0, 4)
sim_res1 (s=1):  3.6561642962057457
sim_res2 (s=0.9):  3.670083190644654
sim_res3 (s=1.1):  3.4780967073146396
sim_res2 < sim_res1 > sim_res3???:  False

x/N:  20.0
VF estimate:  7.733522475912065
new_sol.simulate(x,2000,s=1):  (7.595689852211298, 0.0, 0, 11)
sim_res1 (s=1):  7.595689852211298
sim_res2 (s=0.9):  7.261246871196037
sim_res3 (s=1.1):  2.995732273553991
sim_res2 < sim_res1 > sim_res3???:  True

x/N:  30.0
VF estimate:  12.367707172029462
new_sol.simulate(x,2000,s=1):  (11.970176624943612, 0.0, 0, 25)
sim_res1 (s=1):  11.970176624943612
sim_res2 (s=0.9):  10.653785469684035
sim_res3 (s=1.1):  3.4011973816621555
sim_res2 < sim_res1 > sim_res3???:  True

x/N:  40.0
VF estimate:  17.60609566290688
new_sol.simulate(x,2000,s=1):  (17.94464140515976, 0.6109358370239517, array(8.02006239e+11), 2000)
sim_res1 (s=1):  17.94464140515976
sim_res2 (s=0.9):  13.822567335797062
sim_r

In [ ]:
new_sol.compute_EV(new_sol.XP)

In [ ]:
VF

In [ ]:
new_sol.build_VF()

In [ ]:
u(new_sol.ncgm.xgrid,1,0) + beta*psurvival(a_bar, CP, WP)*new_sol.compute_EV(new_sol.XP)


In [ ]:
new_sol.VF

In [ ]:
xmax = 400
xnew = np.linspace(1,xmax,100)
vf_new = new_sol.build_VF(xnew)

plt.plot(X[X < xmax], VF[X < xmax], 'o', xnew, vf_new)

In [ ]:
xmax = 1000
xnew = np.linspace(1,xmax,100)
xp_new = new_sol.build_XP(xnew)

plt.plot(X[X < xmax], XP[X < xmax], 'o', xnew, xp_new)

In [ ]:
cp_new = new_sol.build_CP(X = xnew)

plt.plot(X[X < xmax], CP[X < xmax], 'o', xnew, cp_new)

In [ ]:
wp_new = new_sol.build_WP(X = xnew)

plt.plot(X[X < xmax], WP[X < xmax], 'o', xnew, wp_new)

In [ ]:
xmax = 1e5
xnew = np.linspace(1,xmax,100)
vf_new = new_sol.build_VF(xnew)

plt.plot(X[X < xmax], VF[X < xmax], 'o', xnew, vf_new)

In [ ]:
cp_new = new_sol.build_CP(X = xnew)

plt.plot(X[X < xmax], CP[X < xmax], 'o', xnew, cp_new)

In [ ]:
wp_new = new_sol.build_WP(X = xnew)

plt.plot(X[X < xmax], WP[X < xmax], 'o', xnew, wp_new)

In [ ]:
xmax = 100
xnew = np.linspace(1,xmax,100)
vf_new = new_sol.build_VF(xnew)

plt.plot(X[X < xmax], VF[X < xmax], 'o', xnew, vf_new)

In [ ]:
cp_new = new_sol.build_CP(X = xnew)

plt.plot(X[X < xmax], CP[X < xmax], 'o', xnew, cp_new)

In [ ]:
wp_new = new_sol.build_WP(X = xnew)

plt.plot(X[X < xmax], WP[X < xmax], 'o', xnew, wp_new)

In [ ]:
new_sol.x_coeffs